In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader


In [4]:
all_data_df = pd.read_csv('all_data.csv')
X = all_data_df.drop(columns=['gold_price_to_predict', 'Date'])

# Extract y
y = all_data_df['gold_price_to_predict']
X_dropped = X.dropna().astype(np.float32)
y_dropped = y[X_dropped.index].to_numpy()[:-1].astype(np.float32)
X_dropped = X_dropped.to_numpy()[:-1]

def create_sequences(data, y, seq_length):
    sequences = []
    y_fit = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])  # Last 30 days as input
        y_fit.append(y[i])
    return np.array(sequences), np.array(y_fit)


sequence_length = 4
X_seqs, y_fit = create_sequences(X_dropped, y, sequence_length)
X_seqs = torch.tensor(X_seqs, dtype=torch.float32)
y_fit = torch.tensor(y_fit, dtype=torch.float32)
X_train, X_test, y_train, y_test = train_test_split(X_seqs, y_fit, test_size=0.2, shuffle=False)


In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]


In [6]:
train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

# DataLoader with batching and shuffling
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        # input_seq: shape (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(input_seq)  # No need to reshape
        predictions = self.linear(lstm_out[:, -1, :])  # Only take the output of the last time step
        return predictions


In [8]:
model = LSTMModel(input_size=X_train.shape[2], hidden_layer_size=100, output_size=1)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
epochs = 150
for epoch in range(epochs):
    total_loss = 0
    for X_batch, y_batch in train_loader:  # Iterate over batches
        model.hidden_cell = (torch.zeros(1, X_batch.size(0), model.hidden_layer_size),
                             torch.zeros(1, X_batch.size(0), model.hidden_layer_size))

        optimizer.zero_grad()

        predictions = model(X_batch)
        loss = loss_function(torch.ravel(predictions), y_batch)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()*y_batch.shape[0]
    print(f'Epoch {epoch} Loss: {loss.item()/len(train_dataset)}')

Epoch 0 Loss: 8.585664216222124
Epoch 1 Loss: 7.9778939044053345
Epoch 2 Loss: 7.891674580509401
Epoch 3 Loss: 8.119593251257106
Epoch 4 Loss: 7.760542775743332
Epoch 5 Loss: 7.415478451574114
Epoch 6 Loss: 6.758885992839965
Epoch 7 Loss: 7.557126557717534
Epoch 8 Loss: 7.261021876366419
Epoch 9 Loss: 6.806498432034324
Epoch 10 Loss: 6.551823998414954
Epoch 11 Loss: 6.7898059172770004
Epoch 12 Loss: 7.073076509209663
Epoch 13 Loss: 6.824816558264102
Epoch 14 Loss: 7.042503313565807
Epoch 15 Loss: 6.158320978629209
Epoch 16 Loss: 6.5452126660198955
Epoch 17 Loss: 5.919405624863358
Epoch 18 Loss: 6.427821057198295
Epoch 19 Loss: 6.205158402109751
Epoch 20 Loss: 6.083119688046567
Epoch 21 Loss: 5.785123370545475
Epoch 22 Loss: 6.373032356799301
Epoch 23 Loss: 5.842989929492785
Epoch 24 Loss: 5.845882040473327
Epoch 25 Loss: 5.500927883827066
Epoch 26 Loss: 5.755364475158505
Epoch 27 Loss: 5.454174580509401
Epoch 28 Loss: 5.551536623442282
Epoch 29 Loss: 5.440575723518802
Epoch 30 Loss: 5.

In [12]:
model.eval()
with torch.no_grad():
    future_price = torch.ravel(model(X_test))
    rmse = np.sqrt(mean_squared_error(y_test, future_price))
    mae = mean_absolute_error(y_test, future_price)

    print(f"Root Mean Square Error: {rmse}")
    print(f"Mean Absolute Error: {mae}")

Root Mean Square Error: 68.819580078125
Mean Absolute Error: 66.1340560913086
